In [1]:
import pickle 

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)
# 저장된 rag_data.pickle을 읽기 모드로 열기

In [2]:
questions = rag_data['questions'][:10]
contexts = rag_data['contexts'][:10]
answers = rag_data['answers'][:10]
len(questions)

10

In [3]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
# evaluator_llm을 LangchainLLMWrapper 클래스로 정의. 이때 모델은 "gpt-4o"를 사용.
# ChatOpenAI: OpenAI의 모델과 상호작용하는 데 사용되는 클래스.
# LangchainLLMWrapper: LLM wrapping에 사용되는 클래스.
# Wrapping: 어떤 객체를 다른 객체로 "포장"하여 원래 객체의 기능 유지하면서 추가적인 처리나 개선 할 수 있도록 하는 것.
# Wrapping을 통해여 LLM이 더 고수준의 기능을 할 수 있도록 함. 예를 들어 평가, 디버깅, 사용자 정의 처리 등등..

d:\학교\기타\학회\HateSlopStudy\rag\RAG-1stWeek\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 평가

### Baseline (w/o RAG)

In [ ]:
from tqdm import tqdm
#tqdm library: progress bar 표현하는 데 사용되는 라이브러리.

from utils import call_openai, get_embeddings, cosine_similarity

predictions = []
# predictions라는 빈 리스트 만듦.
for i in tqdm(range(len(questions))):
    # questions의 길이(10)만큼 tqdm 설정 후 반복문 수행.
    prompt = f"""You are an expert in Finance. Please answer to the question given below.
    
Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    # 생성된 응답을 prediction 변수에 저장
    predictions.append(prediction)
    # 이후 predictions list에 append.

100%|██████████| 10/10 [00:58<00:00,  5.83s/it]


In [6]:
predictions

['글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:\n\n1. **경제 성장 둔화**: 선진국을 중심으로 경제 성장률이 둔화되면서 중앙은행들이 경기 부양을 위해 금리를 낮추는 정책을 지속적으로 시행하고 있습니다. 이는 저금리 현상을 장기화시키는 주요 요인 중 하나입니다.\n\n2. **인구 고령화**: 많은 국가에서 인구 고령화가 진행되면서 소비와 투자 수요가 감소하고 있습니다. 이는 경제 전반의 성장 잠재력을 낮추고, 중앙은행들이 금리를 낮게 유지하도록 압박합니다.\n\n3. **저물가**: 글로벌 경제에서 물가 상승률이 낮게 유지되고 있습니다. 인플레이션이 낮으면 중앙은행들은 금리를 낮게 유지할 수밖에 없습니다. 이는 저금리 현상을 지속시키는 또 다른 요인입니다.\n\n4. **글로벌 금융위기 이후의 정책**: 2008년 글로벌 금융위기 이후, 많은 중앙은행들이 양적 완화(QE)와 같은 비전통적 통화정책을 도입하여 금리를 낮추고 유동성을 공급했습니다. 이러한 정책들은 저금리 환경을 조성하는 데 큰 역할을 했습니다.\n\n5. **안전 자산에 대한 수요 증가**: 글로벌 경제의 불확실성이 증가하면서 투자자들이 안전 자산(예: 국채)에 대한 수요가 증가했습니다. 이는 국채 금리를 낮추는 결과를 초래하고, 전반적인 금리 수준을 낮추는 데 기여합니다.\n\n6. **기술 발전과 생산성 증가**: 기술 발전과 생산성 증가로 인해 기업들이 비용을 절감하고, 이는 물가 상승 압력을 낮추는 결과를 초래합니다. 낮은 물가는 중앙은행들이 금리를 낮게 유지할 수 있는 환경을 제공합니다.\n\n이와 같은 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되고 있습니다.',
 '고수익-고위험 부문으로의 쏠림현상은 투자자들이 높은 수익을 기대하면서 상대적으로 높은 위험을 감수하는 투자 상품이나 자산에 집중하는 현상을 말합니다. 이러한 현상은 여러 가지 요인에 의해 발생할 수 있습니다.\n\n1. **

In [ ]:
from ragas.metrics import FactualCorrectness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate

samples = []
for question, ai_response, ground_truth in zip(questions, predictions, answers):
    # zip()을 통해 questions, predictions, answers 에서 한 아이템 씩 가져와
    #(question, ai_response, ground_truth) 튜플로 저장.
    sample = SingleTurnSample(
        user_input=question,
        response=ai_response,
        reference=ground_truth
    )
    # SingleTurnSample: 평가 위한 샘플 데이터 구조 정의하는 class. question, ai_response, ground_truth로 구성됨.
    samples.append(sample)


dataset = EvaluationDataset(samples)
# samples list를 EvaluationDataset 객체로 변환. 
factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)
#factual_correctness_metric을 FactualCorrectness 객체로 지정
#FactualCorrectness: 생성된 응답이 제공된 컨텍스트와 비교했을 때 정확한 정보를 담고 있는지 평가.

results = evaluate(dataset, metrics=[factual_correctness_metric])
# dataset과 factual_correctness_metric을 비교하여 점수 반환. 이후 results에 저장.
results_df = results.to_pandas()
print(results_df.head())

Evaluating: 100%|██████████| 10/10 [00:35<00:00,  3.59s/it]


                                          user_input  \
0                         글로벌 저금리 현상이 부각된 원인은 무엇인가요?   
1                     고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.   
2  20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...   
3                  2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?   
4                              금융시스템 개혁법은 언제 제정되었나요?   

                                            response  \
0  글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다...   
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 높은 수익을 기대하면서 상대적으로 높...   
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 여러 가지 중요한...   
3  2000년대 이후 글로벌 투자은행(Investment Banks, IB)들은 다양한...   
4  금융시스템 개혁법은 국가마다 다를 수 있습니다. 예를 들어, 미국의 경우 2010년...   

                                           reference  factual_correctness  
0  글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융...                 0.25  
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...                 0.22  
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...                 0.05  
3  2000년대 이후 글로벌 IB들은 

In [57]:
print(results)

{'factual_correctness': 0.1320}


In [58]:
print(questions[0])

글로벌 저금리 현상이 부각된 원인은 무엇인가요?


In [59]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:

1. **경제 성장 둔화**: 글로벌 경제 성장률이 둔화되면서 중앙은행들은 경제를 부양하기 위해 금리를 낮추는 정책을 채택했습니다. 이는 소비와 투자를 촉진하기 위한 조치입니다.

2. **인구 고령화**: 많은 선진국에서 인구 고령화가 진행되면서 저축률이 증가하고, 이는 자본 공급을 늘려 금리를 낮추는 요인으로 작용합니다.

3. **기술 발전**: 기술 발전으로 인해 생산성이 향상되고, 이는 물가 상승 압력을 낮추어 중앙은행들이 금리를 낮게 유지할 수 있는 환경을 조성합니다.

4. **글로벌화**: 글로벌화로 인해 자본이 국경을 넘어 자유롭게 이동하면서, 자본 시장의 통합이 이루어졌습니다. 이는 금리의 국제적 동조화를 촉진하고, 전 세계적으로 금리가 낮아지는 현상을 초래했습니다.

5. **중앙은행의 통화 정책**: 2008년 금융위기 이후 주요 중앙은행들은 양적 완화(QE)와 같은 비전통적 통화 정책을 통해 대규모로 자산을 매입하고, 금리를 낮추는 정책을 지속적으로 시행해 왔습니다.

6. **저물가**: 전 세계적으로 물가 상승률이 낮게 유지되면서, 중앙은행들은 인플레이션 목표를 달성하기 위해 금리를 낮게 유지하는 경향이 있습니다.

이러한 요인들이 복합적으로 작용하면서 글로벌 저금리 현상이 지속되고 있습니다.


In [ ]:
import numpy as np
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity


def retrieve_context(question, contexts):
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')
    # 'text-embedding-3-large'모델을 이용해 question과 contexts의 임베딩 반환.

    similarities = [cosine_similarity(question_embedding, context_embedding) for context_embedding in context_embeddings]
    # 각 question_embedding과 context_embedding 사이의 cosine similarity 계산
    most_relevant_index = np.argmax(similarities)
    # similarity가 가장 큰 인덱스를 반환
    return contexts[most_relevant_index]

predictions = []
for i in tqdm(range(len(questions))):
    context = retrieve_context(questions[i], contexts[i])
    # questions, contexts 리스트에서 가장 similarity가 높은 항목을 프롬프트에 넣음.
    prompt = f"""You are an expert in Finance. Please answer to the question given below. Use information given in Context appropriately.

Context:
{context}

Question:
{questions[i]}
"""
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)
    # 응답 결과를 predictions list에 저장.

100%|██████████| 10/10 [00:54<00:00,  5.48s/it]


In [ ]:
from ragas.metrics import FactualCorrectness
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset
from ragas import evaluate

samples = []
for question, ai_response, ground_truth in zip(questions, predictions, answers):
    sample = SingleTurnSample(
        user_input=question,
        response=ai_response,
        reference=ground_truth
    )
    samples.append(sample)


dataset = EvaluationDataset(samples)
# samples를 EvaluationDataset class로 변환하여 dataset에 저장.
factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)
#factual_correctness_metric을 FactualCorrectness 객체로 지정
#FactualCorrectness: 생성된 응답이 제공된 컨텍스트와 비교했을 때 정확한 정보를 담고 있는지 평가.
results = evaluate(dataset, metrics=[factual_correctness_metric])
# dataset과 factual_correctness_metric을 비교하여 점수 반환. 이후 results에 저장.
results_df = results.to_pandas()
print(results_df.head())

Evaluating: 100%|██████████| 10/10 [00:25<00:00,  2.60s/it]


                                          user_input  \
0                         글로벌 저금리 현상이 부각된 원인은 무엇인가요?   
1                     고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.   
2  20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...   
3                  2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?   
4                              금융시스템 개혁법은 언제 제정되었나요?   

                                            response  \
0  글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 ...   
1  고수익-고위험 부문으로의 쏠림현상은 저금리 기조 하에서 투자자들이 원하는 수익률을 ...   
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 여러 가지 중요한...   
3  2000년대 이후 글로벌 투자은행들(IB)은 여러 전략을 추진하였습니다. 주요 전략...   
4  금융시스템 개혁법에 대한 정보는 주어진 문맥에서 명확하게 언급되지 않았습니다. 그러...   

                                           reference  factual_correctness  
0  글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융...                 0.53  
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...                 0.55  
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...                 0.62  
3  2000년대 이후 글로벌 IB들은 

In [63]:
print(results)

{'factual_correctness': 0.5580}


In [64]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 같은 요소들이 있습니다:

1. **완화적 통화정책**: 2008년 글로벌 금융위기 이후 미국, 유럽, 일본 등 주요국들이 경제를 회복시키기 위해 완화적 통화정책을 시행하였습니다. 이러한 정책은 금리를 낮추고 유동성을 공급하여 경제를 부양하려는 목적이 있었습니다.

2. **경제구조적 요인**: 글로벌 금융위기 이전부터 이어진 경제구조적 요인들도 저금리 현상에 기여하였습니다. 예를 들어, 선진국의 성장세 둔화와 신흥국의 성장으로 인해 글로벌 무역 불균형이 심화되고, 저축과잉현상이 발생하였습니다.

3. **코로나 팬데믹**: 최근 코로나 사태로 인해 경제가 큰 타격을 받았고, 이를 극복하기 위해 각국 중앙은행들이 다시 한 번 완화적 통화정책을 강화하였습니다. 이로 인해 저금리 기조가 더욱 부각되었습니다.

이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 지속되고 있습니다.


In [65]:
print(answers[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의 완화적 통화정책이 있습니다. 둘째, 선진국의 성장세가 둔화되고, 신흥국이 성장하면서 글로벌 무역 불균형이 심화되고 저축과잉현상이 발생했습니다. 셋째, 기술발전에 따른 자본재 가격 하락과 신기술 혁신에 따른 수익률이 낮아져 경제 전체적인 유효투자가 감소했습니다. 넷째, 세계적인 인구증가율 감소 추세는 유효수요 및 자본의 한계생산성과 투자이익 및 투자증가율을 감소시켰습니다. 따라서 이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되었습니다.
